<a href="https://colab.research.google.com/github/ArwaEssa/Automated-Attendance-System/blob/main/Mask_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2,os
import random
from tensorflow.keras.optimizers import Adam 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers
import plotly.graph_objects as go
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16 
from keras.applications.vgg19 import VGG19  
from tensorflow.keras.applications import mobilenet_v2
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score,f1_score, confusion_matrix
from keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Dropout, Flatten,InputLayer
from tensorflow .keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras.applications.inception_v3 import InceptionV3
from sklearn.metrics import accuracy_score

# Uploading the dataset

In [1]:
data_path='/content/drive/MyDrive/Face_mask/AFDB_masked_face_dataset'

In [4]:
CATEGORIES = os.listdir(data_path)
print(CATEGORIES)
print(len(CATEGORIES))

['caizhuoyan', 'chenbailin', 'chenhe', 'chenqiaoen', 'chenweiting', 'dengchao', 'dazhangwei', 'fanbingbing', 'dongjie', 'denglun', 'dilireba', 'duhaitao', 'ditianlin', 'gaoyuanyuan', 'gaoyixiang', 'fengjianyu', 'fangzuming', 'guanxiaotong', 'guojingfei', 'hanxue', 'handongjun', 'houminghao', 'huangtingting', 'huangrihua', 'huangjingyu', 'huangxiaoming', 'huanglei', 'huosiyan', 'huojianhua', 'huozun', 'huangzitao', 'huangzongze', 'huge', 'jiangkaitong', 'jiangxin', 'jianghongbo', 'jianailiang', 'jingtian', 'jindong', 'jingbairan', 'jujingyi', 'jinchen', 'jikejuanyi', 'lilianjie', 'lingengxin', 'lidaimo', 'lifeier', 'linzhiling', 'liuchutian', 'linyun', 'liruotong', 'linyoujia', 'linxinru', 'liuhaoran', 'liuxiang', 'liushishi', 'liutao', 'liuyifei', 'liweijia', 'lixiaolu', 'liyifeng', 'liuxiaoqing', 'lizi', 'luyi', 'luchen', 'liyitong', 'masichun', 'masu', 'make', 'maozijun', 'panyueming', 'mowenwei', 'matianyu', 'qinlan', 'qiaoxin', 'pengyuyan']
76


In [5]:
training_data = []
IMG_SIZE=128
def making_trian_dataset():
    for category in CATEGORIES:
        path = os.path.join(data_path, category) # path to our data
        class_num = CATEGORIES.index(category)# classifcation index
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) # resizing our imgz
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
making_trian_dataset()

In [6]:
print(len(training_data))

441


In [7]:
random.shuffle(training_data)
train_data, labels_data = zip(*training_data)

In [8]:
for Sample in training_data[:10]:
    print(Sample[1])

16
21
15
11
4
43
62
21
9
71


In [9]:
X=[]
y=[]
for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X)#.reshape(-1, IMG_SIZE,IMG_SIZE,3)
y = to_categorical(y, num_classes = 76)

In [10]:
X.shape,y.shape

((441, 128, 128, 3), (441, 76))

In [11]:
nsample,nx,ny,ni=X.shape
X1=X.reshape(nsample,nx*ny*ni)
print(X1.shape)

(441, 49152)


In [12]:
X1_train, X1_val_test, y1_train, y1_val_test = train_test_split(X1, y, test_size=.2, random_state=77)
X1_val, X1_test, y1_val, y1_test = train_test_split(X1_val_test, y1_val_test, test_size=0.5, random_state=77)
print(f"\nTraining data: {X1_train.shape},  labels: {y1_train.shape}")
print(f"Validation data: {X1_val.shape},  labels: {y1_val.shape}")
print(f"Testing data: {X1_test.shape},  labels: {y1_test.shape}")


Training data: (352, 49152),  labels: (352, 76)
Validation data: (44, 49152),  labels: (44, 76)
Testing data: (45, 49152),  labels: (45, 76)


In [18]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=.2, random_state=77)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=77)
print(f"\nTraining data: {X_train.shape},  labels: {y_train.shape}")
print(f"Validation data: {X_val.shape},  labels: {y_val.shape}")
print(f"Testing data: {X_test.shape},  labels: {y_test.shape}")


Training data: (352, 128, 128, 3),  labels: (352, 76)
Validation data: (44, 128, 128, 3),  labels: (44, 76)
Testing data: (45, 128, 128, 3),  labels: (45, 76)


In [15]:
simple_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=X1_train.shape[1:]),
    keras.layers.Dense(units=15, activation="relu"),
    keras.layers.Dense(units=10, activation='softmax'),
    keras.layers.Dense(units=5, activation="relu"),
    keras.layers.Dense(units=76, activation='softmax'),

])
simple_model.compile("sgd", loss="categorical_crossentropy", metrics=["accuracy"])
simple_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 15)                737295    
                                                                 
 dense_5 (Dense)             (None, 10)                160       
                                                                 
 dense_6 (Dense)             (None, 5)                 55        
                                                                 
 dense_7 (Dense)             (None, 76)                456       
                                                                 
Total params: 737,966
Trainable params: 737,966
Non-trainable params: 0
_________________________________________________________________


In [16]:
simple_model.fit(X1_train, y1_train,validation_data=(X1_val,y1_val) ,batch_size=64, epochs=15)

Epoch 1/15
6/6 [==============================] - 2s 45ms/step - loss: 4.3404 - accuracy: 0.0142 - val_loss: 4.3418 - val_accuracy: 0.0000e+00
Epoch 2/15
6/6 [==============================] - 0s 8ms/step - loss: 4.3429 - accuracy: 0.0142 - val_loss: 4.3415 - val_accuracy: 0.0000e+00
Epoch 3/15
6/6 [==============================] - 0s 8ms/step - loss: 4.3423 - accuracy: 0.0142 - val_loss: 4.3414 - val_accuracy: 0.0000e+00
Epoch 4/15
6/6 [==============================] - 0s 8ms/step - loss: 4.3417 - accuracy: 0.0142 - val_loss: 4.3412 - val_accuracy: 0.0000e+00
Epoch 5/15
6/6 [==============================] - 0s 8ms/step - loss: 4.3411 - accuracy: 0.0142 - val_loss: 4.3410 - val_accuracy: 0.0000e+00
Epoch 6/15
6/6 [==============================] - 0s 8ms/step - loss: 4.3405 - accuracy: 0.0142 - val_loss: 4.3409 - val_accuracy: 0.0000e+00
Epoch 7/15
6/6 [==============================] - 0s 8ms/step - loss: 4.3400 - accuracy: 0.0142 - val_loss: 4.3407 - val_accuracy: 0.0000e+00
Epoch

In [17]:
num_classes=76
model_0 = Sequential()

model_0.add(InputLayer(input_shape=(128,128,3)))

model_0.add(Conv2D(32, 3, activation='relu'))   
model_0.add(MaxPooling2D())

model_0.add(Conv2D(32, 3, activation='relu'))   
model_0.add(MaxPooling2D())

model_0.add(Conv2D(32, 3, activation='relu'))  
model_0.add(MaxPooling2D())

model_0.add(Flatten())
model_0.add(Dense(128, activation='relu'))
model_0.add(Dense(num_classes , activation = 'softmax'))   

model_0.compile(optimizer='adam', 
                loss="categorical_crossentropy", 
                metrics=['accuracy'])
model_0.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)      

In [19]:
history_0 = model_0.fit(X_train, y_train, validation_data=(X_val,y_val),epochs=10,batch_size=64)   


Epoch 1/10
6/6 [==============================] - 8s 75ms/step - loss: 47.1326 - accuracy: 0.0170 - val_loss: 10.4332 - val_accuracy: 0.0000e+00
Epoch 2/10
6/6 [==============================] - 0s 16ms/step - loss: 6.0223 - accuracy: 0.0312 - val_loss: 4.7098 - val_accuracy: 0.0455
Epoch 3/10
6/6 [==============================] - 0s 16ms/step - loss: 4.0143 - accuracy: 0.0909 - val_loss: 4.4687 - val_accuracy: 0.0227
Epoch 4/10
6/6 [==============================] - 0s 16ms/step - loss: 3.5066 - accuracy: 0.2159 - val_loss: 4.5851 - val_accuracy: 0.0682
Epoch 5/10
6/6 [==============================] - 0s 16ms/step - loss: 2.7393 - accuracy: 0.4261 - val_loss: 4.8721 - val_accuracy: 0.0455
Epoch 6/10
6/6 [==============================] - 0s 16ms/step - loss: 1.9396 - accuracy: 0.5881 - val_loss: 5.4890 - val_accuracy: 0.1364
Epoch 7/10
6/6 [==============================] - 0s 16ms/step - loss: 1.2048 - accuracy: 0.7330 - val_loss: 5.3910 - val_accuracy: 0.0909
Epoch 8/10
6/6 [=====

In [20]:
model_1 = Sequential()

model_1.add(InputLayer(input_shape=(128, 128,3)))

model_1.add(Conv2D(32, 3, activation='relu'))    
model_1.add(MaxPooling2D())

model_1.add(Conv2D(64, 3, activation='relu'))   
model_1.add(MaxPooling2D())

model_1.add(Conv2D(128, 3, activation='relu'))    
model_1.add(MaxPooling2D())

model_1.add(Flatten())
model_1.add(Dense(512, activation='relu')) 
model_1.add(Dropout(.05))   # replace 0.20
model_1.add(Dense(num_classes , activation = 'softmax'))  

model_1.compile(optimizer='Adagrad',loss="categorical_crossentropy",metrics=['accuracy'])
# model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model_1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 128)     

In [22]:
callbacks=[
        keras.callbacks.EarlyStopping(patience=8, verbose=1, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=.5, patience=3, verbose=1),
    ]
history_1=model_1.fit(X_train, y_train, batch_size=64, epochs=30, validation_data=(X_val,y_val), callbacks=[callbacks])

Epoch 1/30
6/6 [==============================] - 0s 38ms/step - loss: 2.3407 - accuracy: 0.4744 - val_loss: 4.4921 - val_accuracy: 0.1364 - lr: 0.0010
Epoch 2/30
6/6 [==============================] - 0s 27ms/step - loss: 1.9153 - accuracy: 0.5824 - val_loss: 4.3964 - val_accuracy: 0.1818 - lr: 0.0010
Epoch 3/30
6/6 [==============================] - 0s 22ms/step - loss: 1.4819 - accuracy: 0.7188 - val_loss: 4.5168 - val_accuracy: 0.1364 - lr: 0.0010
Epoch 4/30
6/6 [==============================] - 0s 22ms/step - loss: 1.1665 - accuracy: 0.8125 - val_loss: 5.0871 - val_accuracy: 0.1136 - lr: 0.0010
Epoch 5/30
4/6 [===================>..........] - ETA: 0s - loss: 1.1843 - accuracy: 0.7812
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
6/6 [==============================] - 0s 22ms/step - loss: 1.0964 - accuracy: 0.8068 - val_loss: 4.5443 - val_accuracy: 0.1364 - lr: 0.0010
Epoch 6/30
6/6 [==============================] - 0s 26ms/step - loss: 0.6671 -

In [23]:
model_1_ = Sequential()

model_1_.add(InputLayer(input_shape=(128, 128,3)))

model_1_.add(Conv2D(32, 3, activation='relu'))   
model_1_.add(MaxPooling2D())

model_1_.add(Conv2D(64, 3, activation='relu'))   
model_1_.add(MaxPooling2D())

model_1_.add(Conv2D(128, 3, activation='relu'))    
model_1_.add(MaxPooling2D())

model_1_.add(Flatten())
model_1_.add(Dense(128, activation='relu'))    #512
model_1_.add(Dense(num_classes , activation = 'softmax'))  

model_1_.compile(optimizer='adam', 
                loss="categorical_crossentropy", 
                metrics=['accuracy'])
model_1_.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 14, 14, 128)     

In [24]:
model_1_= model_1_.fit(X_train, y_train, validation_data=(X_val,y_val),epochs=10,batch_size=64)


Epoch 1/10
6/6 [==============================] - 1s 56ms/step - loss: 95.6631 - accuracy: 0.0114 - val_loss: 5.6600 - val_accuracy: 0.0227
Epoch 2/10
6/6 [==============================] - 0s 21ms/step - loss: 5.1467 - accuracy: 0.0227 - val_loss: 4.5124 - val_accuracy: 0.0227
Epoch 3/10
6/6 [==============================] - 0s 20ms/step - loss: 4.3409 - accuracy: 0.0284 - val_loss: 4.4280 - val_accuracy: 0.0455
Epoch 4/10
6/6 [==============================] - 0s 20ms/step - loss: 4.2409 - accuracy: 0.0625 - val_loss: 4.5728 - val_accuracy: 0.0227
Epoch 5/10
6/6 [==============================] - 0s 21ms/step - loss: 4.1446 - accuracy: 0.0597 - val_loss: 4.4179 - val_accuracy: 0.1364
Epoch 6/10
6/6 [==============================] - 0s 20ms/step - loss: 3.9549 - accuracy: 0.0966 - val_loss: 4.4790 - val_accuracy: 0.0455
Epoch 7/10
6/6 [==============================] - 0s 20ms/step - loss: 3.6391 - accuracy: 0.1619 - val_loss: 4.6830 - val_accuracy: 0.0682
Epoch 8/10
6/6 [==========

In [25]:
model_2 = Sequential()

model_2.add(InputLayer(input_shape=(128, 128,3)))

model_2.add(Conv2D(32, 3, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))    
model_2.add(MaxPooling2D())

model_2.add(Conv2D(64, 3, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))   
model_2.add(MaxPooling2D())

model_2.add(Conv2D(128, 3, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))    
model_2.add(MaxPooling2D())

model_2.add(Flatten())
model_2.add(Dense(512, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))     
model_2.add(Dense(num_classes , activation = 'softmax'))  

model_2.compile(optimizer='adam', 
                loss="categorical_crossentropy", 
                metrics=['accuracy'])
model_2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 30, 30, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 14, 14, 128)     

In [26]:
model_2.fit(X_train, y_train, validation_data=(X_val,y_val),epochs=10)


Epoch 1/10
11/11 [==============================] - 1s 38ms/step - loss: 61.5524 - accuracy: 0.0227 - val_loss: 5.2960 - val_accuracy: 0.0227
Epoch 2/10
11/11 [==============================] - 0s 15ms/step - loss: 4.6214 - accuracy: 0.1875 - val_loss: 5.6929 - val_accuracy: 0.0682
Epoch 3/10
11/11 [==============================] - 0s 15ms/step - loss: 3.0366 - accuracy: 0.5710 - val_loss: 6.5739 - val_accuracy: 0.0227
Epoch 4/10
11/11 [==============================] - 0s 15ms/step - loss: 1.7015 - accuracy: 0.8438 - val_loss: 7.8898 - val_accuracy: 0.1364
Epoch 5/10
11/11 [==============================] - 0s 15ms/step - loss: 1.3551 - accuracy: 0.9290 - val_loss: 11.4872 - val_accuracy: 0.0000e+00
Epoch 6/10
11/11 [==============================] - 0s 15ms/step - loss: 1.2701 - accuracy: 0.9602 - val_loss: 11.1708 - val_accuracy: 0.0909
Epoch 7/10
11/11 [==============================] - 0s 15ms/step - loss: 1.2922 - accuracy: 0.9545 - val_loss: 8.7220 - val_accuracy: 0.0682
Epoch 

In [27]:
model_3 = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3)) 
 
for layer in model_3.layers:
    layer.trainable = False    

x = model_3.output
x = Flatten()(x) 
x = Dense(100, activation='relu')(x) 
x = Dense(50, activation='relu')(x)
predictions = Dense(76, activation='softmax')(x) 

model_3 = Model(inputs=model_3.input, outputs=predictions)
model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_3.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_7[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)             

In [28]:
model_3.fit(X_train, y_train, validation_data=(X_val,y_val),epochs=10)


Epoch 1/10
11/11 [==============================] - 4s 114ms/step - loss: 4.8058 - accuracy: 0.0199 - val_loss: 4.4915 - val_accuracy: 0.0227
Epoch 2/10
11/11 [==============================] - 0s 17ms/step - loss: 4.2629 - accuracy: 0.0426 - val_loss: 4.5001 - val_accuracy: 0.0455
Epoch 3/10
11/11 [==============================] - 0s 17ms/step - loss: 4.0264 - accuracy: 0.0653 - val_loss: 4.3054 - val_accuracy: 0.0227
Epoch 4/10
11/11 [==============================] - 0s 17ms/step - loss: 3.7145 - accuracy: 0.1364 - val_loss: 4.4614 - val_accuracy: 0.0000e+00
Epoch 5/10
11/11 [==============================] - 0s 17ms/step - loss: 3.4174 - accuracy: 0.1648 - val_loss: 4.5943 - val_accuracy: 0.0227
Epoch 6/10
11/11 [==============================] - 0s 16ms/step - loss: 3.1040 - accuracy: 0.2500 - val_loss: 4.5573 - val_accuracy: 0.0227
Epoch 7/10
11/11 [==============================] - 0s 17ms/step - loss: 2.7141 - accuracy: 0.3551 - val_loss: 4.6524 - val_accuracy: 0.0227
Epoch 8/

In [29]:
model_4 = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))  

# Freeze convolutional layers
for layer in model_4.layers:
    layer.trainable = False  

NN_transfer = Sequential(
                        [InputLayer(input_shape=(128,128,3)),model_4,
                         Flatten(),  # should be fine , or add layers
                         Dense(64, activation='relu'),   
                         Dense(32, activation='relu'),   # 2 dense is must bcuz VGG16 model Conv2D twice and Maxpooling -> get a lot more features
                         Dense(76, activation='softmax')]
                       )

NN_transfer.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'],)
model_4.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                              

In [30]:
result_NN_t = NN_transfer.fit(X_train,y_train, validation_data=(X_val,y_val), epochs=10)


Epoch 1/10
11/11 [==============================] - 3s 103ms/step - loss: 9.0381 - accuracy: 0.0085 - val_loss: 4.7249 - val_accuracy: 0.0455
Epoch 2/10
11/11 [==============================] - 0s 27ms/step - loss: 4.5215 - accuracy: 0.0341 - val_loss: 4.3303 - val_accuracy: 0.0455
Epoch 3/10
11/11 [==============================] - 0s 27ms/step - loss: 4.2998 - accuracy: 0.0369 - val_loss: 4.3304 - val_accuracy: 0.0455
Epoch 4/10
11/11 [==============================] - 0s 27ms/step - loss: 4.2372 - accuracy: 0.0312 - val_loss: 4.2863 - val_accuracy: 0.0000e+00
Epoch 5/10
11/11 [==============================] - 0s 27ms/step - loss: 4.1308 - accuracy: 0.0511 - val_loss: 4.2375 - val_accuracy: 0.0000e+00
Epoch 6/10
11/11 [==============================] - 0s 27ms/step - loss: 4.0158 - accuracy: 0.0540 - val_loss: 4.2060 - val_accuracy: 0.0455
Epoch 7/10
11/11 [==============================] - 0s 27ms/step - loss: 3.9222 - accuracy: 0.0966 - val_loss: 4.1648 - val_accuracy: 0.0455
Epoc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')